# Preprocessing

Preprocessing notebook

In [6]:
import numpy as np
import pandas as pd

pd.options.plotting.backend = "plotly"

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.metrics import concordance_index_ipcw
from sksurv.util import Surv

df = pd.read_csv("../../data/clinical_train.csv")
df_eval = pd.read_csv("../../data/clinical_val.csv")
maf_df = pd.read_csv("../../data/molecular_train.csv")
maf_eval = pd.read_csv("../../data/molecular_val.csv")
target_df = pd.read_csv("../../data/target_train.csv")

In [2]:
target = ['OS_YEARS', 'OS_STATUS']

# Drop rows where 'OS_YEARS' is NaN if conversion caused any issues
target_df.dropna(subset=target, inplace=True)
target_df['OS_YEARS'] = pd.to_numeric(target_df['OS_YEARS'], errors='coerce')
target_df['OS_STATUS'] = target_df['OS_STATUS'].astype(bool)

# Select features
features = ['ID', 'BM_BLAST', 'WBC', 'ANC', 'MONOCYTES', 'HB', 'PLT', 'CYTOGENETICS']

# Create the survival data format
X = df.loc[df['ID'].isin(target_df['ID']), features]
X_eval = df_eval.loc[:, features]
y = Surv.from_dataframe('OS_STATUS', 'OS_YEARS', target_df)


In [7]:
df.columns

Index(['ID', 'CENTER', 'BM_BLAST', 'WBC', 'ANC', 'MONOCYTES', 'HB', 'PLT',
       'CYTOGENETICS'],
      dtype='object')

## XGBoost-based Imputation

Create an XGBoost imputer to fill missing values using OS_YEARS as the prediction target. The imputer will be trained on complete cases and used to fill missing values in both training and evaluation datasets.

In [8]:
num_cols = X.select_dtypes("number").columns
print(f"Numerical columns are: {num_cols.values}")

o = len(X)
X = X[X[num_cols].isna().sum(axis=1) <= 4]
print(f"Dropping every row with more than 4 missing numerical values. {o - len(X)} rows dropped, {len(X)} remaining.")


Numerical columns are: ['BM_BLAST' 'WBC' 'ANC' 'MONOCYTES' 'HB' 'PLT']
Dropping every row with more than 4 missing numerical values. 70 rows dropped, 3103 remaining.


In [9]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
print(f"Fitting RobustScaler for numerical columns ({num_cols.values}) on training data, transforming training and evaluation data.")
X[num_cols] = scaler.fit_transform(X[num_cols])
X_eval[num_cols] = scaler.transform(X_eval[num_cols])


Fitting RobustScaler for numerical columns (['BM_BLAST' 'WBC' 'ANC' 'MONOCYTES' 'HB' 'PLT']) on training data, transforming training and evaluation data.


In [10]:
import optuna
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score

# Identify columns with missing values (only numerical columns)
missing_cols = [col for col in num_cols if X[col].isnull().any()]
print(f"Columns with missing values: {missing_cols}")

# For each column with missing values, train separate XGBoost model
xgb_imputers = {}

for col in missing_cols:
    print(f"\nTraining XGBoost imputer for {col}...")

    # Get rows where current column is not null
    col_complete_mask = X[col].notnull()
    X_train_col = X.loc[col_complete_mask, num_cols].drop(columns=[col])
    y_train_col = X.loc[col_complete_mask, col]

    # Handle any remaining missing values in other columns for training
    X_train_col = X_train_col.fillna(X_train_col.median())


    # Define objective function for Optuna
    def objective(trial):
        params = {
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.3, log=True),
            'n_estimators': trial.suggest_int('n_estimators', 50, 500),
            'subsample': trial.suggest_float('subsample', 0.2, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.2, 1.0),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            'gamma': trial.suggest_float('gamma', 0, 5),
            'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),
            'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True),
            'random_state': 42,
            'objective': 'reg:squarederror'
        }

        model = XGBRegressor(**params)
        scores = cross_val_score(model, X_train_col, y_train_col,
                                 cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
        return -scores.mean()


    # Run Optuna optimization
    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(objective, n_trials=250, show_progress_bar=True)

    print(f"Best parameters for {col}: {study.best_params}")
    print(f"Best CV score for {col}: {study.best_value:.4f}")

    # Train final model with best parameters
    best_params = study.best_params
    best_params['random_state'] = 42
    best_params['objective'] = 'reg:squarederror'

    best_model = XGBRegressor(**best_params)
    best_model.fit(X_train_col, y_train_col)

    # Store best estimator
    xgb_imputers[col] = best_model

[I 2025-11-12 21:56:45,519] A new study created in memory with name: no-name-ab369a43-aa11-4e12-bc16-a03f4e4b81cc


Columns with missing values: ['BM_BLAST', 'WBC', 'ANC', 'MONOCYTES', 'HB', 'PLT']

Training XGBoost imputer for BM_BLAST...


  0%|          | 0/250 [00:00<?, ?it/s]

[I 2025-11-12 21:56:49,818] Trial 0 finished with value: 1.2973025106320804 and parameters: {'max_depth': 5, 'learning_rate': 0.22648248189516848, 'n_estimators': 380, 'subsample': 0.6789267873576292, 'colsample_bytree': 0.3248149123539492, 'min_child_weight': 2, 'gamma': 0.2904180608409973, 'reg_alpha': 0.08499808989182997, 'reg_lambda': 0.0006440507553993703}. Best is trial 0 with value: 1.2973025106320804.
[I 2025-11-12 21:56:53,392] Trial 1 finished with value: 1.1430977571058478 and parameters: {'max_depth': 8, 'learning_rate': 0.001124579825911934, 'n_estimators': 487, 'subsample': 0.8659541126403374, 'colsample_bytree': 0.36987128854262097, 'min_child_weight': 2, 'gamma': 0.9170225492671691, 'reg_alpha': 2.716051144654844e-06, 'reg_lambda': 0.00015777981883364995}. Best is trial 1 with value: 1.1430977571058478.
[I 2025-11-12 21:56:56,633] Trial 2 finished with value: 1.044160892083807 and parameters: {'max_depth': 6, 'learning_rate': 0.005265139631677754, 'n_estimators': 325, '

[I 2025-11-12 21:57:40,981] A new study created in memory with name: no-name-42443491-3b7c-4912-8043-763946f653c8


...

[I 2025-11-12 21:59:31,284] Trial 0 finished with value: 0.6996443065548001 and parameters: {'max_depth': 5, 'learning_rate': 0.22648248189516848, 'n_estimators': 380, 'subsample': 0.6789267873576292, 'colsample_bytree': 0.3248149123539492, 'min_child_weight': 2, 'gamma': 0.2904180608409973, 'reg_alpha': 0.08499808989182997, 'reg_lambda': 0.0006440507553993703}. Best is trial 0 with value: 0.6996443065548001.
[I 2025-11-12 21:59:31,475] Trial 1 finished with value: 0.6028392307386576 and parameters: {'max_depth': 8, 'learning_rate': 0.001124579825911934, 'n_estimators': 487, 'subsample': 0.8659541126403374, 'colsample_bytree': 0.36987128854262097, 'min_child_weight': 2, 'gamma': 0.9170225492671691, 'reg_alpha': 2.716051144654844e-06, 'reg_lambda': 0.00015777981883364995}. Best is trial 1 with value: 0.6028392307386576.
[I 2025-11-12 21:59:31,584] Trial 2 finished with value: 0.5749275255824294 and parameters: {'max_depth': 6, 'learning_rate': 0.005265139631677754, 'n_estimators': 325, 

[I 2025-11-12 21:59:59,889] A new study created in memory with name: no-name-2ca87ecf-2ca5-4f49-ace6-51b310b31f6f


[I 2025-11-12 21:59:59,730] Trial 249 finished with value: 0.5619389467461805 and parameters: {'max_depth': 4, 'learning_rate': 0.00857823853075669, 'n_estimators': 408, 'subsample': 0.5602143530647107, 'colsample_bytree': 0.9411219309575252, 'min_child_weight': 5, 'gamma': 1.9275060733314207, 'reg_alpha': 1.0039579834732024e-06, 'reg_lambda': 0.0030217570207211894}. Best is trial 237 with value: 0.5605076952794756.
Best parameters for HB: {'max_depth': 4, 'learning_rate': 0.011623802563927916, 'n_estimators': 408, 'subsample': 0.5964064852872966, 'colsample_bytree': 0.9810544027074726, 'min_child_weight': 5, 'gamma': 2.0411630622978727, 'reg_alpha': 4.078506948050003e-06, 'reg_lambda': 0.0015103449134969533}
Best CV score for HB: 0.5605

Training XGBoost imputer for PLT...


  0%|          | 0/250 [00:00<?, ?it/s]

[I 2025-11-12 22:00:00,053] Trial 0 finished with value: 0.9109762691417911 and parameters: {'max_depth': 5, 'learning_rate': 0.22648248189516848, 'n_estimators': 380, 'subsample': 0.6789267873576292, 'colsample_bytree': 0.3248149123539492, 'min_child_weight': 2, 'gamma': 0.2904180608409973, 'reg_alpha': 0.08499808989182997, 'reg_lambda': 0.0006440507553993703}. Best is trial 0 with value: 0.9109762691417911.
[I 2025-11-12 22:00:00,255] Trial 1 finished with value: 0.7950016855772122 and parameters: {'max_depth': 8, 'learning_rate': 0.001124579825911934, 'n_estimators': 487, 'subsample': 0.8659541126403374, 'colsample_bytree': 0.36987128854262097, 'min_child_weight': 2, 'gamma': 0.9170225492671691, 'reg_alpha': 2.716051144654844e-06, 'reg_lambda': 0.00015777981883364995}. Best is trial 1 with value: 0.7950016855772122.
[I 2025-11-12 22:00:00,386] Trial 2 finished with value: 0.7277604211901961 and parameters: {'max_depth': 6, 'learning_rate': 0.005265139631677754, 'n_estimators': 325, 

In [11]:
def impute_missing_values(df, imputers, numerical_cols, dataset_name="dataset"):
    """
    Apply XGBoost imputation to missing values in a dataframe.
    
    Parameters:
    - df: DataFrame to impute
    - imputers: Dictionary of trained XGBoost imputers per column
    - numerical_cols: List of numerical column names to impute
    - dataset_name: Name for logging purposes
    
    Returns:
    - DataFrame with imputed values
    """
    missing_cols = [col for col in imputers.keys() if col in df.columns]

    for col in missing_cols:
        if df[col].isnull().sum() == 0:
            continue

        print(f"\nImputing {col} in {dataset_name}...")

        missing_mask = df[col].isnull()
        X_pred = df.loc[missing_mask, numerical_cols].drop(columns=[col])
        X_pred = X_pred.fillna(df[numerical_cols].drop(columns=[col]).median())

        predictions = imputers[col].predict(X_pred)
        df.loc[missing_mask, col] = predictions

        print(f"  Imputed {missing_mask.sum()} missing values")

    print(f"\nMissing values in {dataset_name} after imputation:")
    print(df[numerical_cols].isnull().sum())

    return df

In [12]:
X = impute_missing_values(X, xgb_imputers, num_cols, "X")
X_eval = impute_missing_values(X_eval, xgb_imputers, num_cols, "X_eval")



Imputing BM_BLAST in X...
  Imputed 27 missing values

Imputing WBC in X...
  Imputed 159 missing values

Imputing ANC in X...
  Imputed 71 missing values

Imputing MONOCYTES in X...
  Imputed 474 missing values

Imputing HB in X...
  Imputed 5 missing values

Imputing PLT in X...
  Imputed 16 missing values

Missing values in X after imputation:
BM_BLAST     0
WBC          0
ANC          0
MONOCYTES    0
HB           0
PLT          0
dtype: int64

Imputing BM_BLAST in X_eval...
  Imputed 115 missing values

Imputing WBC in X_eval...
  Imputed 112 missing values

Imputing ANC in X_eval...
  Imputed 141 missing values

Imputing MONOCYTES in X_eval...
  Imputed 883 missing values

Imputing HB in X_eval...
  Imputed 111 missing values

Imputing PLT in X_eval...
  Imputed 115 missing values

Missing values in X_eval after imputation:
BM_BLAST     0
WBC          0
ANC          0
MONOCYTES    0
HB           0
PLT          0
dtype: int64


## Enhanced Mutation Features

Computing mutation count (Nmut) along with VAF statistics (average, std, max) and length statistics (average, std, max) for both training and evaluation datasets.

Looking at the user request and the selected cell, I need to apply RobustScaler to the new mutation features (Nmut, VAF_avg, VAF_std, VAF_max, LEN_avg, LEN_std, LEN_max) that are computed in the `compute_mutation_features` function.

The task is to:
1. Apply RobustScaler to the newly computed mutation features after merging them with the dataframe
2. Fit the scaler on training data and transform both training and evaluation data



In [59]:
maf_df["CHR"].value_counts()[maf_df["CHR"].value_counts() > 90].index

Index(['4', '2', '17', 'X', '20', '21', '12', '7', '5', '1', '11', '15', '19',
       '18', '9', '3', '16'],
      dtype='object', name='CHR')

In [61]:
maf_df["EFFECT"].unique()

array(['non_synonymous_codon', 'stop_gained', 'splice_site_variant',
       'frameshift_variant', 'inframe_codon_loss', 'inframe_codon_gain',
       'initiator_codon_change', '2KB_upstream_variant',
       'complex_change_in_transcript', '3_prime_UTR_variant', 'stop_lost',
       'inframe_variant', 'synonymous_codon', 'stop_retained_variant',
       'ITD', 'PTD'], dtype=object)

In [43]:
pd.get_dummies(maf_df, columns=["EFFECT"])

,ID,CHR,START,END,REF,ALT,GENE,PROTEIN_CHANGE,VAF,DEPTH,...,EFFECT_inframe_codon_gain,EFFECT_inframe_codon_loss,EFFECT_inframe_variant,EFFECT_initiator_codon_change,EFFECT_non_synonymous_codon,EFFECT_splice_site_variant,EFFECT_stop_gained,EFFECT_stop_lost,EFFECT_stop_retained_variant,EFFECT_synonymous_codon
0,P100000,11,119149248.0,119149248.0,G,A,CBL,p.C419Y,0.0830,1308.0,...,False,False,False,False,True,False,False,False,False,False
1,P100000,5,131822301.0,131822301.0,G,T,IRF1,p.Y164*,0.0220,532.0,...,False,False,False,False,False,False,True,False,False,False
2,P100000,3,77694060.0,77694060.0,G,C,ROBO2,p.?,0.4100,876.0,...,False,False,False,False,False,True,False,False,False,False
3,P100000,4,106164917.0,106164917.0,G,T,TET2,p.R1262L,0.4300,826.0,...,False,False,False,False,True,False,False,False,False,False
4,P100000,2,25468147.0,25468163.0,ACGAAGAGGGGGTGTTC,A,DNMT3A,p.E505fs*141,0.0898,942.0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10930,P131472,NaN,NaN,NaN,NaN,NaN,MLL,MLL_PTD,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
10931,P131505,NaN,NaN,NaN,NaN,NaN,MLL,MLL_PTD,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
10932,P131816,NaN,NaN,NaN,NaN,NaN,MLL,MLL_PTD,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
10933,P132717,NaN,NaN,NaN,NaN,NaN,MLL,MLL_PTD,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False


Looking at the user request, I need to add CHR count columns for the chromosomes specified in the list: ['4', '2', '17', 'X', '20', '21', '12', '7', '5', '1', '11', '15', '19', '18', '9', '3', '16'].

Currently, the code only has `CHR_X_count`. I need to add similar count columns for all the other chromosomes in the list.

Here's the modified cell code:



In [62]:
def compute_mutation_features(maf_df, X_df):
    """Compute mutation statistics and merge with features dataframe."""
    # Compute length for each mutation
    maf_df['LEN'] = maf_df['END'] - maf_df['START'] + 1
    maf_df['DELLEN'] = maf_df['LEN'] - maf_df['REF'].apply(lambda x: len(str(x)))

    # Compute aggregated statistics
    tmp = maf_df.groupby('ID').agg(
        Nmut=('ID', 'size'),
        VAF_avg=('VAF', 'mean'),
        VAF_std=('VAF', 'std'),
        VAF_max=('VAF', 'max'),
        LEN_avg=('LEN', 'mean'),
        LEN_max=('LEN', 'max'),
        DELLEN_sum=('DELLEN', 'sum'),
        DEPTH_avg=('DEPTH', 'mean'),
        DEPTH_std=('DEPTH', 'std'),
        DEPTH_max=('DEPTH', 'max'),
        DEPTH_min=('DEPTH', 'min'),
        CHR_nunique=('CHR', 'nunique'),
        CHR_4_count=('CHR', lambda x: (x == '4').sum()),
        CHR_2_count=('CHR', lambda x: (x == '2').sum()),
        CHR_17_count=('CHR', lambda x: (x == '17').sum()),
        CHR_X_count=('CHR', lambda x: (x == 'X').sum()),
        CHR_20_count=('CHR', lambda x: (x == '20').sum()),
        CHR_21_count=('CHR', lambda x: (x == '21').sum()),
        CHR_12_count=('CHR', lambda x: (x == '12').sum()),
        CHR_7_count=('CHR', lambda x: (x == '7').sum()),
        CHR_5_count=('CHR', lambda x: (x == '5').sum()),
        CHR_1_count=('CHR', lambda x: (x == '1').sum()),
        CHR_11_count=('CHR', lambda x: (x == '11').sum()),
        CHR_15_count=('CHR', lambda x: (x == '15').sum()),
        CHR_19_count=('CHR', lambda x: (x == '19').sum()),
        CHR_18_count=('CHR', lambda x: (x == '18').sum()),
        CHR_9_count=('CHR', lambda x: (x == '9').sum()),
        CHR_3_count=('CHR', lambda x: (x == '3').sum()),
        CHR_16_count=('CHR', lambda x: (x == '16').sum()),
        EFFECT_nunique=('EFFECT', 'nunique'),
        EFFECT_FV_count=('EFFECT', lambda x: (x == 'frameshift_variant').sum()),
        EFFECT_SG_count=('EFFECT', lambda x: (x == 'stop_gained').sum()),
        EFFECT_NS_count=('EFFECT', lambda x: (x == 'non_synonymous_codon').sum()),
    ).reset_index()

    # Fill NaN std values with 0 (occurs when patient has only 1 mutation)
    tmp[['VAF_std', 'DEPTH_std']] = tmp[['VAF_std', 'DEPTH_std']].fillna(0)

    # Merge with X and fill missing values with 0
    X_w_mutation = X_df.merge(tmp, on='ID', how='left').fillna({
        'Nmut': 0, 'VAF_avg': 0, 'VAF_std': 0, 'VAF_max': 0,
        'LEN_avg': 0, 'LEN_max': 0, 'DELLEN_sum': 0,
        'DEPTH_avg': 0, 'DEPTH_std': 0, 'DEPTH_max': 0, 'DEPTH_min': 0,
        'CHR_nunique': 0, 'CHR_4_count': 0, 'CHR_2_count': 0, 'CHR_17_count': 0,
        'CHR_X_count': 0, 'CHR_20_count': 0, 'CHR_21_count': 0, 'CHR_12_count': 0,
        'CHR_7_count': 0, 'CHR_5_count': 0, 'CHR_1_count': 0, 'CHR_11_count': 0,
        'CHR_15_count': 0, 'CHR_19_count': 0, 'CHR_18_count': 0, 'CHR_9_count': 0,
        'CHR_3_count': 0, 'CHR_16_count': 0, 'EFFECT_nunique': 0, 'EFFECT_FV_count': 0,
        'EFFECT_SG_count': 0, 'EFFECT_NS_count': 0
    })

    return X_w_mutation


# Apply function to training and evaluation data
X_w_mutation = compute_mutation_features(maf_df, X)
X_eval_w_mutation = compute_mutation_features(maf_eval, X_eval)

# Apply RobustScaler to the new mutation features
mutation_features = ['Nmut', 'VAF_avg', 'VAF_std', 'VAF_max', 'LEN_avg', 'LEN_max', 'DELLEN_sum',
                     'DEPTH_avg', 'DEPTH_std', 'DEPTH_max', 'DEPTH_min', 'CHR_nunique',
                     'CHR_4_count', 'CHR_2_count', 'CHR_17_count', 'CHR_X_count', 'CHR_20_count',
                     'CHR_21_count', 'CHR_12_count', 'CHR_7_count', 'CHR_5_count', 'CHR_1_count',
                     'CHR_11_count', 'CHR_15_count', 'CHR_19_count', 'CHR_18_count', 'CHR_9_count',
                     'CHR_3_count', 'CHR_16_count', 'EFFECT_nunique', 'EFFECT_FV_count', 'EFFECT_SG_count', 'EFFECT_NS_count']
mutation_scaler = RobustScaler()
print(
    f"Fitting RobustScaler for mutation features ({mutation_features}) on training data, transforming training and evaluation data.")
X_w_mutation[mutation_features] = mutation_scaler.fit_transform(X_w_mutation[mutation_features])
X_eval_w_mutation[mutation_features] = mutation_scaler.transform(X_eval_w_mutation[mutation_features])


Fitting RobustScaler for mutation features (['Nmut', 'VAF_avg', 'VAF_std', 'VAF_max', 'LEN_avg', 'LEN_max', 'DELLEN_sum', 'DEPTH_avg', 'DEPTH_std', 'DEPTH_max', 'DEPTH_min', 'CHR_nunique', 'CHR_4_count', 'CHR_2_count', 'CHR_17_count', 'CHR_X_count', 'CHR_20_count', 'CHR_21_count', 'CHR_12_count', 'CHR_7_count', 'CHR_5_count', 'CHR_1_count', 'CHR_11_count', 'CHR_15_count', 'CHR_19_count', 'CHR_18_count', 'CHR_9_count', 'CHR_3_count', 'CHR_16_count', 'EFFECT_nunique', 'EFFECT_FV_count', 'EFFECT_SG_count', 'EFFECT_NS_count']) on training data, transforming training and evaluation data.


In [63]:
X_w_mutation

,ID,BM_BLAST,WBC,ANC,MONOCYTES,HB,PLT,CYTOGENETICS,Nmut,VAF_avg,...,CHR_15_count,CHR_19_count,CHR_18_count,CHR_9_count,CHR_3_count,CHR_16_count,EFFECT_nunique,EFFECT_FV_count,EFFECT_SG_count,EFFECT_NS_count
0,P132697,1.571429,-0.3250,-0.666667,0.513219,-0.807692,-0.024316,"46,xy,del(20)(q12)[2]/46,xy[18]",2.000000,-0.131147,...,0.0,0.0,0.0,0.0,0.0,1.0,0.5,2.0,1.0,4.0
1,P132698,-0.285714,0.8250,0.148148,-0.419907,0.730769,-0.492401,"46,xx",0.000000,-0.042407,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,-1.0
2,P116889,1.714286,-0.1000,0.037037,-0.419907,1.730769,-0.255319,"46,xy,t(3;3)(q25;q27)[8]/46,xy[12]",0.000000,-1.015868,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,2.0,0.0
3,P132699,-0.285714,-0.0500,-0.037037,-0.419907,-0.307692,-0.279635,"46,xy,del(3)(q26q27)[15]/46,xy[5]",2.666667,-0.307681,...,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,6.0
4,P132700,0.428571,30.9750,2.851852,0.824261,0.538462,0.437690,"46,xx,t(3;9)(p13;q22)[10]/46,xx[10]",-0.666667,0.788078,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.5,0.0,0.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3098,P121826,-0.285714,-0.4000,-0.362963,-0.264386,0.192308,-0.273556,"47~49,xy,+21,+21,+22[cp11]/46,xy[10]",0.333333,-0.475365,...,0.0,0.0,1.0,0.0,0.0,0.0,-0.5,-1.0,0.0,3.0
3099,P121827,-0.214286,1.0000,0.244444,0.124417,0.615385,-0.504559,"44,xx,inv(2)(p25q31),-4,del(5)(q12q33),-18[20]",-0.333333,0.602168,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.5,-1.0,0.0,1.0
3100,P121830,-0.428571,-0.5750,-0.537037,-0.124417,-0.115385,-0.224924,"46,xy,del(20)(q11.2q13.1)[4]/45,xy,idem,-7[16]",1.000000,-0.160025,...,0.0,0.0,0.0,1.0,0.0,0.0,0.5,0.0,2.0,2.0
3101,P121853,0.285714,-0.6825,-0.603704,-0.404355,0.653846,-0.127660,"46,xx,del(1)(p34)[5]/45,xx,sl,-18[12]/46,xx,sd...",0.333333,-0.411797,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.5,-1.0,0.0,3.0


# Processing Cytogenetics

sorte de "rules engine" basé sur la littérature clinique AML pour extraire des features pronostiques clés depuis l’ISCN. On détecte

* **Monosomal karyotype (MK)**: au moins 2 monosomies autosomiques, ou 1 monosomie autosomique + au moins une anomalie structurale. C’est franchement mauvais pronostic. ([PubMed Central][1])
* **Complex karyotype**: typiquement ≥ 3 anomalies cytogénétiques indépendantes, pronostic défavorable. ([Cancer Network][2])
* **Chromosomes 5 et 7**: perte de 5 ou 7, ou del(5q)/del(7q), classés défavorables dans ELN 2022. ([ASH Publications][3])
* **+8 (trisomie 8)**: très fréquente, plutôt risque intermédiaire en AML. ([MDPI][4])
* **Translocations favorables classiques**: t(8;21), inv(16)/t(16;16); APL t(15;17) est à part. On les isole comme features car elles portent un signal fort. ([ASH Publications][3])
* **Proportions clonales** via les crochets `[n]` pour calculer la part de métaphases portant une anomalie donnée. La sémantique des notations vient de l’ISCN. ([PubMed][5])

Pour chaque karyotype, on construit:

* `has_any_abnormality`, `n_events`, `n_chromosomes_altered`
* `has_minus5_or_del5q`, `has_minus7_or_del7q`, `has_plus8`
* `has_t_8_21`, `has_inv16_or_t_16_16`, `has_t_15_17`
* `is_complex_karyotype`, `is_monosomal_karyotype`
* `total_metaphases` et des **proportions clonales**: `prop_any_abnormal`, `prop_adverse_5_7`, `prop_plus8`, etc.
* Option: `eln_like_flag_adverse_cyto` basé ici sur 5/7 ou MK ou complex.

[1]: https://pmc.ncbi.nlm.nih.gov/articles/PMC3069222/?utm_source=chatgpt.com "Acute myeloid leukemia with monosomal karyotype at the ..."
[2]: https://www.cancernetwork.com/view/unfavorable-complex-and-monosomal-karyotypes-most-challenging-forms-acute-myeloid-leukemia?utm_source=chatgpt.com "Unfavorable, Complex, and Monosomal Karyotypes"
[3]: https://ashpublications.org/blood/article/140/12/1345/485817/Diagnosis-and-management-of-AML-in-adults-2022?utm_source=chatgpt.com "Diagnosis and management of AML in adults - ASH Publications"
[4]: https://www.mdpi.com/2072-6694/13/22/5679?utm_source=chatgpt.com "Risk Stratification, Measurable Residual Disease, and ..."
[5]: https://pubmed.ncbi.nlm.nih.gov/34839499/?utm_source=chatgpt.com "[Introduction and interpretation of the updated contents of ..."


In [64]:
import re
from typing import List, Dict

# --- REGEX patterns ---
_ISCN_EVENT_RE = re.compile(r'(del|dup|inv|ins|i|t|add|der)\s*\(', re.IGNORECASE)
_MONOSOMY_RE   = re.compile(r'(?<![pq])-(\d{1,2}|X|Y)(?![pq])', re.IGNORECASE)
_TRISOMY_RE    = re.compile(r'(?<![pq])\+(\d{1,2}|X|Y)(?![pq])', re.IGNORECASE)
_CHR_NUM_RE    = re.compile(r'(?<![pq])(\d{1,2}|X|Y)(?![pq])', re.IGNORECASE)

# specific adverse/favorable
_MINUS5_OR_DEL5Q_RE = re.compile(r'-(?:5)(?![pq])|del\s*\(\s*5\s*\)\s*\(\s*q', re.IGNORECASE)
_MINUS7_OR_DEL7Q_RE = re.compile(r'-(?:7)(?![pq])|del\s*\(\s*7\s*\)\s*\(\s*q', re.IGNORECASE)
_PLUS8_RE           = re.compile(r'\+8(?![pq])', re.IGNORECASE)
_T_8_21_RE          = re.compile(r't\s*\(\s*8\s*;\s*21\s*\)', re.IGNORECASE)
_INV16_OR_T_16_16_RE= re.compile(r'(inv\s*\(\s*16\s*\)|t\s*\(\s*16\s*;\s*16\s*\))', re.IGNORECASE)
_T_15_17_RE         = re.compile(r't\s*\(\s*15\s*;\s*17\s*\)', re.IGNORECASE)
_STRUCTURAL_RE      = re.compile(r'(del|dup|inv|ins|i|t|add|der)\s*\(', re.IGNORECASE)

# --- helpers ---
def _split_clones(karyo: str) -> List[str]:
    return [c.strip() for c in str(karyo).split('/') if c.strip()]

def _extract_metaphases(clone: str) -> int:
    m = re.search(r'\[(\d+)\]', clone)
    return int(m.group(1)) if m else 0

def _count_events(clone: str) -> int:
    n_struct = len(_ISCN_EVENT_RE.findall(clone))
    n_mono   = len(_MONOSOMY_RE.findall(clone))
    n_tri    = len(_TRISOMY_RE.findall(clone))
    n_mono_minusY = len(re.findall(r'(?<![pq])-(?:Y)(?![pq])', clone, flags=re.IGNORECASE))
    return n_struct + n_tri + max(n_mono - n_mono_minusY, 0)

def _chromosomes_altered(clone: str) -> int:
    nums = set()
    for m in _MONOSOMY_RE.finditer(clone):
        nums.add(m.group(1).upper())
    for m in _TRISOMY_RE.finditer(clone):
        nums.add(m.group(1).upper())
    for ev in re.finditer(r'(del|dup|inv|ins|i|t|add|der)\s*\(([^)]+)\)', clone, flags=re.IGNORECASE):
        for x in re.split(r'[;,\s]+', ev.group(2)):
            if _CHR_NUM_RE.fullmatch(x.strip()):
                nums.add(x.strip().upper())
    nums.discard('Y')
    return len(nums)

def _has_structural(clone: str) -> bool:
    return bool(_STRUCTURAL_RE.search(clone))

def _autosomic_monosomies(clone: str) -> List[int]:
    return [int(m.group(1)) for m in _MONOSOMY_RE.finditer(clone) if m.group(1).upper() not in ('X','Y')]

def _is_monosomal_karyotype(karyo: str) -> bool:
    clones = _split_clones(karyo)
    autosomal_monosomies = set()
    any_struct = False
    for c in clones:
        autosomal_monosomies.update(_autosomic_monosomies(c))
        any_struct = any_struct or _has_structural(c)
    return (len(autosomal_monosomies) >= 2) or (len(autosomal_monosomies) >= 1 and any_struct)

def _is_complex_karyotype(karyo: str) -> bool:
    clones = _split_clones(karyo)
    total_events = 0
    for c in clones:
        c_wo_minusY = re.sub(r'(?<![pq])-(?:Y)(?![pq])', '', c, flags=re.IGNORECASE)
        total_events += _count_events(c_wo_minusY)
    return total_events >= 3

def _clone_flags(clone: str) -> Dict[str, bool]:
    return {
        'minus5_or_del5q': bool(_MINUS5_OR_DEL5Q_RE.search(clone)),
        'minus7_or_del7q': bool(_MINUS7_OR_DEL7Q_RE.search(clone)),
        'plus8':           bool(_PLUS8_RE.search(clone)),
        't_8_21':          bool(_T_8_21_RE.search(clone)),
        'inv16_or_t_16_16':bool(_INV16_OR_T_16_16_RE.search(clone)),
        't_15_17':         bool(_T_15_17_RE.search(clone)),
        'has_structural':  _has_structural(clone),
        'events_count':    _count_events(clone),
        'chrs_altered':    _chromosomes_altered(clone),
        'has_any_abn':     bool(_ISCN_EVENT_RE.search(clone) or _MONOSOMY_RE.search(clone) or _TRISOMY_RE.search(clone)),
    }

# --- main featurizer ---
def add_cytogenetics_features(df: pd.DataFrame, col: str = "CYTOGENETICS") -> pd.DataFrame:
    rows = []
    for k in df[col]:
        clones = _split_clones(k)
        clone_info = []
        total_meta_known = 0
        for c in clones:
            n_meta = _extract_metaphases(c)
            flags = _clone_flags(c)
            clone_info.append((c, n_meta, flags))
            total_meta_known += n_meta

        any_abn = any(f['has_any_abn'] for _,_,f in clone_info)
        n_events = sum(f['events_count'] for _,_,f in clone_info)
        n_chrs   = max([f['chrs_altered'] for _,_,f in clone_info] + [0])

        has_minus5_or_del5q = any(f['minus5_or_del5q'] for _,_,f in clone_info)
        has_minus7_or_del7q = any(f['minus7_or_del7q'] for _,_,f in clone_info)
        has_plus8           = any(f['plus8'] for _,_,f in clone_info)
        has_t_8_21          = any(f['t_8_21'] for _,_,f in clone_info)
        has_inv16_or_t_16_16= any(f['inv16_or_t_16_16'] for _,_,f in clone_info)
        has_t_15_17         = any(f['t_15_17'] for _,_,f in clone_info)

        is_mk  = _is_monosomal_karyotype(k)
        is_ck  = _is_complex_karyotype(k)
        eln_like_adverse = bool(is_mk or is_ck or has_minus5_or_del5q or has_minus7_or_del7q)

        def _prop(cond_fn):
            if total_meta_known == 0:
                return 0.0
            pos = sum(n_meta for _, n_meta, f in clone_info if n_meta and cond_fn(f))
            return pos / total_meta_known if total_meta_known else 0.0

        rows.append({
            'has_any_abnormality': int(any_abn),
            'n_events': int(n_events),
            'n_chromosomes_altered': int(n_chrs),
            'has_minus5_or_del5q': int(has_minus5_or_del5q),
            'has_minus7_or_del7q': int(has_minus7_or_del7q),
            'has_plus8': int(has_plus8),
            'has_t_8_21': int(has_t_8_21),
            'has_inv16_or_t_16_16': int(has_inv16_or_t_16_16),
            'has_t_15_17': int(has_t_15_17),
            'is_monosomal_karyotype': int(is_mk),
            'is_complex_karyotype': int(is_ck),
            'eln_like_flag_adverse_cyto': int(eln_like_adverse),
            'total_metaphases': int(total_meta_known),
            'prop_any_abnormal': float(_prop(lambda f: f['has_any_abn'])),
            'prop_adverse_5_7': float(_prop(lambda f: f['minus5_or_del5q'] or f['minus7_or_del7q'])),
            'prop_plus8': float(_prop(lambda f: f['plus8'])),
            'prop_favorable_core': float(_prop(lambda f: f['t_8_21'] or f['inv16_or_t_16_16'])),
        })
    features_df = pd.DataFrame(rows, index=df.index)
    return pd.concat([df.copy(), features_df], axis=1).drop(columns=[col])


In [65]:
X_enhanced = add_cytogenetics_features(X_w_mutation)
X_eval_enhanced = add_cytogenetics_features(X_eval_w_mutation)

# Identify nearly constant cytogenetics features (95% or more the same value)
cytogenetics_features = ['has_any_abnormality', 'n_events', 'n_chromosomes_altered',
                         'has_minus5_or_del5q', 'has_minus7_or_del7q', 'has_plus8',
                         'has_t_8_21', 'has_inv16_or_t_16_16', 'has_t_15_17',
                         'is_monosomal_karyotype', 'is_complex_karyotype',
                         'eln_like_flag_adverse_cyto', 'total_metaphases',
                         'prop_any_abnormal', 'prop_adverse_5_7', 'prop_plus8',
                         'prop_favorable_core']

nearly_constant_features = []
for col in cytogenetics_features:
    value_counts = X_enhanced[col].value_counts()
    if len(value_counts) > 0:
        max_proportion = value_counts.iloc[0] / len(X_enhanced)
        if max_proportion >= 0.95:
            nearly_constant_features.append(col)
            print(f"Removing {col}: {max_proportion:.2%} of values are {value_counts.index[0]}")

# Remove nearly constant features
X_enhanced = X_enhanced.drop(columns=nearly_constant_features)
X_eval_enhanced = X_eval_enhanced.drop(columns=nearly_constant_features)

# Keep only non-constant features for scaling
cytogenetics_features = [f for f in cytogenetics_features if f not in nearly_constant_features]

# Apply RobustScaler to the remaining cytogenetics features
cytogenetics_scaler = RobustScaler()
print(f"Fitting RobustScaler for cytogenetics features on training data, transforming training and evaluation data.")
X_enhanced[cytogenetics_features] = cytogenetics_scaler.fit_transform(X_enhanced[cytogenetics_features])
X_eval_enhanced[cytogenetics_features] = cytogenetics_scaler.transform(X_eval_enhanced[cytogenetics_features])


Removing has_t_8_21: 100.00% of values are 0
Removing has_inv16_or_t_16_16: 100.00% of values are 0
Removing has_t_15_17: 99.97% of values are 0
Removing prop_favorable_core: 100.00% of values are 0.0
Fitting RobustScaler for cytogenetics features on training data, transforming training and evaluation data.


In [66]:
df_enhanced = X_enhanced.merge(target_df, on='ID', how='left')
df_eval_enhanced = X_eval_enhanced

df_enhanced.to_csv('../../data/train_enhanced.csv', index=False)
df_eval_enhanced.to_csv('../../data/eval_enhanced.csv', index=False)

In [ ]:
# Gene feature engineering : keeping the top 50 genes in train. For every gene not appearing in val => delete from train.
def add_gene_features(df_clinical_enhanced, df_molecular, top_genes_list=None):
    if top_genes_list is None:
        top_genes_list = df_molecular['GENE'].value_counts().head(50).index.tolist()
        print(f"Top genes references: {top_genes_list[:5]}")
    df_filtered = df_molecular[df_molecular['GENE'].isin(top_genes_list)].copy()
    gene_pivot = pd.crosstab(df_filtered['ID'], df_filtered['GENE'])
    gene_pivot = (gene_pivot > 0).astype(int)
    gene_pivot.columns = [f'Gene_{col}' for col in gene_pivot.columns]
    df_final = df_clinical_enhanced.merge(gene_pivot, on='ID', how='left')

    new_cols = [c for c in df_final.columns if c.startswith('Gene_')]
    df_final[new_cols] = df_final[new_cols].fillna(0)
    return df_final, top_genes_list

mol_train_raw = pd.read_csv("../../data/molecular_train.csv")
mol_val_raw = pd.read_csv("../../data/molecular_val.csv")

df_train_pivot, top_genes_ref = add_gene_features(df_enhanced, mol_train_raw, top_genes_list=None)
df_val_pivot, _ = add_gene_features(df_eval_enhanced, mol_val_raw, top_genes_list=top_genes_ref)

genes_in_train = set([c for c in df_train_pivot.columns if c.startswith('Gene_')])
genes_in_val   = set([c for c in df_val_pivot.columns if c.startswith('Gene_')])
genes_to_keep = genes_in_train.intersection(genes_in_val)
print(f"Kept : {len(genes_to_keep)}")

cols_train_clinical = set([c for c in df_train_pivot.columns if not c.startswith('Gene_')])
cols_val_clinical   = set([c for c in df_val_pivot.columns if not c.startswith('Gene_')])
common_clinical = cols_train_clinical.intersection(cols_val_clinical)

final_features = list(common_clinical) + list(genes_to_keep)
df_val_pivot = df_val_pivot[final_features]

df_train_pivot.to_csv('../../data/train_pivot.csv', index=False)
df_val_pivot.to_csv('../../data/eval_pivot.csv', index=False)